In [1]:
## Create embeddings and save to vector store

from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path=".env", override=True)
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)


In [2]:
from typing import List, Dict , TypedDict
from typing import Annotated, Literal, TypedDict
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain.schema import Document
from pydantic import BaseModel,Field
import openai
from langsmith.wrappers import wrap_openai
from langsmith import traceable

# Auto-trace LLM calls in-context
client = wrap_openai(openai.Client())


In [3]:
from typing import List, Dict , TypedDict
from typing import Annotated, Literal, TypedDict
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage,ToolMessage
from langgraph.graph import MessagesState, StateGraph
from langchain_core.tools import tool

@tool
def generate_email(no_of_leaves: int, total_enchashment: float, purpose: str) -> str:
    """
    Returns an email with the following information about the employee 
        1. No of leaves
        2. Total encashment amount
        2. Purpose of the email
    The target recepient of the email is an HR Manager of the department

    Parameters:
    no_of_leaves (int): No of leaves 
    total_encashment (float) : Total value of the leaves
    purpose (str): Purpose of the email

    Returns:
    str :  This is an email generated
    
    """

    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    prompt_fmt = f"""You are email writing assistant.
                    Please write a short email to a HR manager with the following information.
                    Leaves : {no_of_leaves}
                    Total Encashment value : {total_enchashment}
                    Purpose : {purpose}
                    Please include the date on which the email is written.
                """
    llm_response = llm.invoke(prompt_fmt) 
    return (llm_response)


In [ ]:
from langgraph.prebuilt import create_react_agent

email_agent = create_react_agent(
    model=llm,
    tools=[generate_email],
    name="email_agent",
    prompt="You are an agent that writes an email to a Hr practitioner"
)

In [7]:

# app = email_agent.compile()
result = email_agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Write an email to encash 15 earned leaves with a total valuation of 32000 INR to the HR requesting to process immediately?"                        
        }
    ]
})

print(result)


{'messages': [HumanMessage(content='Write an email to encash 15 earned leaves with a total valuation of 32000 INR to the HR requesting to process immediately?', additional_kwargs={}, response_metadata={}, id='d6cd2e93-1055-4b75-a3e0-49184237d949'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ucurTbBEcKu43ExIEcFUzmcF', 'function': {'arguments': '{"no_of_leaves":15,"total_enchashment":32000,"purpose":"Request for immediate processing of encashment of earned leaves"}', 'name': 'generate_email'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 204, 'total_tokens': 244, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C56yZStxTbJSOAGFA3ClKn4UZPSsl', 'service_tier': 

In [8]:


for msg in result['messages']:    
    msg.pretty_print()



================================ Human Message =================================

Write an email to encash 15 earned leaves with a total valuation of 32000 INR to the HR requesting to process immediately?
================================== Ai Message ==================================
Name: email_agent
Tool Calls:
  generate_email (call_ucurTbBEcKu43ExIEcFUzmcF)
 Call ID: call_ucurTbBEcKu43ExIEcFUzmcF
  Args:
    no_of_leaves: 15
    total_enchashment: 32000
    purpose: Request for immediate processing of encashment of earned leaves
================================= Tool Message =================================
Name: generate_email

content="Subject: Request for Immediate Processing of Earned Leave Encashment\n\nDear [HR Manager's Name],\n\nI hope this message finds you well. I am writing to request the immediate processing of my earned leave encashment. As of today, [Date], I have a total of 15 earned leaves available for encashment. The total encashment value amounts to 32,000.0.\n

In [9]:
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
import ast
import json
from langchain_core.messages.modifier import RemoveMessage

for message in result["messages"]:
    if isinstance(message, AIMessage):
        ai_message = message.content
        print(f"AI Message: {ai_message}")
        

     

AI Message: 
AI Message: I have drafted the email requesting the immediate processing of encashment for 15 earned leaves with a total valuation of 32,000 INR. 

---

Subject: Request for Immediate Processing of Earned Leave Encashment

Dear [HR Manager's Name],

I hope this message finds you well. I am writing to request the immediate processing of my earned leave encashment. As of today, [Date], I have a total of 15 earned leaves available for encashment. The total encashment value amounts to 32,000.0.

I would appreciate it if you could expedite this process at your earliest convenience. Please let me know if you need any further information or documentation from my side to facilitate this request.

Thank you for your attention to this matter.

Best regards,

[Your Name]  
[Your Position]  
[Your Contact Information]

--- 

You can personalize it with your details before sending it to the HR Manager. Let me know if you need any further assistance!
